In [1]:
import sys
sys.path.append('/home/myk/min0/')
sys.path.append('/home/myk/min0/traffic-classification/DataRepresentation')
import numpy as np
import pandas as pd
import torch
import LSTM
import Label2Int as pp 
import random

/home/myk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/myk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/myk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/myk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

In [3]:
x = np.load('/home/myk/min0/x_tuple2token_900.npy')
y = pp.getlabel("/home/myk/min0/metadata.csv")


0 is arkio
1 is bigballers
2 is bigscreen
3 is blaston
4 is campfire
5 is epicrollercoasters
6 is firsthand
7 is firsttouch
8 is fitxr
9 is godsofgravity
10 is gorillatag
11 is gunraiders
12 is horizonworkrooms
13 is horizonworlds
14 is hyperdash
15 is immersed
16 is netflix
17 is roblox
18 is vrchat
19 is youtube


In [4]:
print(x.shape)
num_index = np.max(x)
print(num_index)

(3180, 900)
8373


In [5]:

input_size = 200
output_size = 20
hidden_dim = 100
num_layers = 4
batch = 128
lr = 0.001
epoch = 100

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
# LSTM model 가져오기 
model = LSTM.LSTM_withEmbed(num_index+1, input_size, hidden_dim,output_size,num_layers)
model = torch.nn.DataParallel(model).to(device)

In [8]:
print(model)


DataParallel(
  (module): LSTM_withEmbed(
    (embedding): Embedding(8374, 150)
    (lstm): LSTM(150, 100, num_layers=3, batch_first=True, dropout=0.22, bidirectional=True)
    (fc): Linear(in_features=100, out_features=20, bias=True)
  )
)


In [9]:
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=lr)

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optim,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)


In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,shuffle=True, random_state=1)

print(x_train.shape)
print(y_train.shape)

(2544, 900)
(2544,)


In [11]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

train_dataset = CustomDataset(x_train,y_train)
test_dataset = CustomDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=True,drop_last=True)

In [12]:


best_loss = 0.1 
for i in range(epoch):
    for j,[data,label] in enumerate(train_loader):
        label = label.type(torch.LongTensor)
        x = data.to(device)
        y = label.to(device)
        outputs = model.forward(x)
        # 손실 계산
        loss = criterion(outputs, y)
        
        # 역전파 및 가중치 업데이트
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        
        if(loss<best_loss):
            best_loss = loss
    if (i+1) % 10 == 0:
        print(f'epoch {(i+1)} :  Loss: {loss.item()}')



epoch 10 :  Loss: 0.5887731313705444
epoch 20 :  Loss: 0.44134578108787537
epoch 30 :  Loss: 0.2097795158624649
epoch 40 :  Loss: 0.12459764629602432
epoch 50 :  Loss: 0.153349831700325
epoch 60 :  Loss: 0.11895999312400818
epoch 70 :  Loss: 0.0626799538731575
epoch 80 :  Loss: 0.14212104678153992
epoch 90 :  Loss: 0.069939024746418
epoch 100 :  Loss: 0.048635490238666534


In [14]:
model.eval()

total =0
correct=0
incorrect_num_correct = np.zeros(120)
incorrect_num_answer = np.zeros(120)
with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y= label.to(device)
        output = model.forward(x)
        
        # torch.max함수는 (최댓값,index)를 반환 
        _,output_index = torch.max(output,1)
        
        # 전체 개수 += 라벨의 개수
        total += label.size(0)
        
        # 도출한 모델의 index와 라벨이 일치하면 correct에 개수 추가
        correct += (output_index == y).sum().float()
        
        for i in range(len(y)):
            if y[i]!=output_index[i]:
                k = int(y[i].cpu().numpy())
                incorrect_num_correct[k] += 1
                incorrect_num_answer[int(output_index[i].cpu().numpy())]+=1

    
    # 정확도 도출
    print("Accuracy of Test Data: {}%".format(100*correct/total))
   

Accuracy of Test Data: 91.9921875%
